In [1]:
"""
Quickstart to Tensorboard
The dataset is using MNIST handwritten dataset.

author: Jiankai Wang
"""

'\nQuickstart to Tensorboard\nThe dataset is using MNIST handwritten dataset.\n\nauthor: Jiankai Wang\n'

In [1]:
from __future__ import print_function
import os
import tensorflow as tf
print("Tensorflow Version: {}".format(tf.__version__))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow Version: 1.9.0


# Prepare

## Load MNIST dataset

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/mnist_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/mnist_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Parameters / Model

## hyper-parameters

In [3]:
full_training = True
learning_rate = 1e-2
training_epochs = 100 if full_training else 1
batch_size = 64 if full_training else 32
display_step = 1
log_path = os.path.join("/","tmp","tensorboard_log","mnist_example")

## network

In [4]:
def conv2d(input, weight_shape, bias_shape):
    in_count = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=(2/in_count)**0.5)
    bias_init = tf.constant_initializer(value=0)
    weight = tf.get_variable("W", weight_shape, initializer=weight_init)
    bias = tf.get_variable("b", bias_shape, initializer=bias_init)
    conv = tf.nn.conv2d(input, weight, strides=[1, 1, 1, 1], padding="SAME")
    return tf.nn.relu(tf.nn.bias_add(conv, bias))

In [5]:
def pool2d(input, k=2):
    return tf.nn.max_pool(value=input, ksize=[1,k,k,1], strides=[1,k,k,1], padding="SAME")

In [6]:
def dense(input, weight_shape, bias_shape):
    in_count = weight_shape[0] * weight_shape[1]
    weight_init = tf.random_normal_initializer(stddev=(2/in_count)**0.5)
    bias_init = tf.constant_initializer(value=0)
    weight = tf.get_variable("W", weight_shape, initializer=weight_init)
    bias = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.matmul(input, weight)
    return tf.nn.relu(tf.nn.bias_add(logits, bias))

In [7]:
def inference(input, keep_prob=0.5):
    x = tf.reshape(input, [-1, 28, 28, 1])
    
    with tf.variable_scope("hidden_1"):
        conv_1 = conv2d(x, [3, 3, 1, 32], [32])         # 28 x 28 x 32
        pool_1 = pool2d(conv_1, 2)                      # 14 x 14 x 32
    
    with tf.variable_scope("hidden_2"):
        conv_2 = conv2d(pool_1, [3, 3, 32, 64], [64])   # 14 x 14 x 64
        pool_2 = pool2d(conv_2, 2)                      # 7 x 7 x 64
        
    with tf.variable_scope("fc"):
        pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
        fc_1 = dense(pool_2_flat, [7 * 7 * 64, 1024], [1024])
        
        # dropout
        fc_1_dropout = tf.nn.dropout(fc_1, keep_prob=keep_prob)
        
    with tf.variable_scope("output"):
        output = dense(fc_1_dropout, [1024, 10], [10])
        
    return output

# Learning

## Target

In [8]:
def loss(output, y):
    """
    output: the logits value from inference
    y: the labeling data
    """
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    return loss

In [9]:
def training(loss, global_step):
    """
    loss: the loss value
    global_step: the global training step index
    """
    tf.summary.scalar("loss", loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads, global_step=global_step)
    return grads, apply_grads

In [10]:
def evaluate(output, y):
    """
    output: the logits value from inference
    y: the labeling data
    """
    compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
    tf.summary.scalar("eval", accuracy)
    return accuracy

## Training

In [11]:
print("""
Run 'tensorboard --logdir=/tmp/tensorboard_log/' to monitor the training process.
""")


Run 'tensorboard --logdir=/tmp/tensorboard_log/' to monitor the training process.



In [12]:
with tf.Graph().as_default():
    with tf.variable_scope("mlp"):
        x = tf.placeholder("float", [None, 784])  # x is batch input
        y = tf.placeholder("float", [None, 10])   # y is output for 10 classification
        keep_prob = tf.placeholder(tf.float32)

        output = inference(x, keep_prob=keep_prob)   # get the inference result
        loss_val = loss(output=output, y=y)   # get the loss
        global_step = tf.Variable(0, name="global_step", trainable=False)   # training step
        train_grads, train_opt = training(loss=loss_val, global_step=global_step)   # training body
        eval_opt = evaluate(output=output, y=y)   # evaluation result
        
        # show all training variable info
        # may cause summary name error
        # INFO:tensorflow:Summary name mlp/hidden_1/W:0 is illegal; using mlp/hidden_1/W_0 instead.
        for var in tf.trainable_variables():
            tf.summary.histogram(var.name, var)
            
        # show grads info
        for grad, var in train_grads:
            tf.summary.histogram(var.name + '/gradient', grad)

        init_var = tf.global_variables_initializer()
        summary_opt = tf.summary.merge_all()   # merge all summaries
        saver = tf.train.Saver()   # for saving checkpoints

        with tf.Session() as sess:
            
            summary_writer = tf.summary.FileWriter(log_path, graph=sess.graph)   # write the summary 
            sess.run(init_var)   # initialize all variables

            for epoch in range(training_epochs):
                avg_loss = 0.
                total_batch = int(mnist.train.num_examples / batch_size)

                for idx in range(total_batch):
                    batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)   # get the batch data

                    feed_dict_data = {x: batch_x, y: batch_y, keep_prob: 0.5}
                    grads, _ = sess.run([train_grads, train_opt], feed_dict=feed_dict_data)   # run training

                    batch_loss = sess.run(loss_val, feed_dict=feed_dict_data)
                    avg_loss += batch_loss / total_batch   # calculate the average loss

                if epoch % display_step == 0:
                    # record log

                    feed_dict_val_data = {x: mnist.validation.images, y: mnist.validation.labels, keep_prob: 1.0}
                    acc = sess.run(eval_opt, feed_dict=feed_dict_val_data)   # calculate the accuracy

                    print("Epoch: {}, Accuracy: {}, Vaildation Error: {}".format(epoch+1, round(acc,2), round(1-acc,2)))
                    tf.summary.scalar("validation_accuracy", acc)  

                    summary_str = sess.run(summary_opt, feed_dict=feed_dict_val_data)
                    summary_writer.add_summary(summary_str, sess.run(global_step))   # write out the summary

                    saver.save(sess, os.path.join(log_path, "model-checkpoint"), global_step=global_step)

            print("Training finishing.")

            feed_dict_test_data = {x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}
            acc = sess.run(eval_opt, feed_dict=feed_dict_test_data)   # test result
            print("Test Accuracy:",acc)

INFO:tensorflow:Summary name mlp/hidden_1/W:0 is illegal; using mlp/hidden_1/W_0 instead.
INFO:tensorflow:Summary name mlp/hidden_1/b:0 is illegal; using mlp/hidden_1/b_0 instead.
INFO:tensorflow:Summary name mlp/hidden_2/W:0 is illegal; using mlp/hidden_2/W_0 instead.
INFO:tensorflow:Summary name mlp/hidden_2/b:0 is illegal; using mlp/hidden_2/b_0 instead.
INFO:tensorflow:Summary name mlp/fc/W:0 is illegal; using mlp/fc/W_0 instead.
INFO:tensorflow:Summary name mlp/fc/b:0 is illegal; using mlp/fc/b_0 instead.
INFO:tensorflow:Summary name mlp/output/W:0 is illegal; using mlp/output/W_0 instead.
INFO:tensorflow:Summary name mlp/output/b:0 is illegal; using mlp/output/b_0 instead.
INFO:tensorflow:Summary name mlp/hidden_1/W:0/gradient is illegal; using mlp/hidden_1/W_0/gradient instead.
INFO:tensorflow:Summary name mlp/hidden_1/b:0/gradient is illegal; using mlp/hidden_1/b_0/gradient instead.
INFO:tensorflow:Summary name mlp/hidden_2/W:0/gradient is illegal; using mlp/hidden_2/W_0/gradie

KeyboardInterrupt: 